In [1]:
import numpy as np
import torch
from torch.utils import data

def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, torch.reshape(y, (-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[-0.9971,  0.5188],
         [-1.7573,  1.7265],
         [ 0.0892, -0.8751],
         [ 1.1947,  0.4706],
         [-1.2170,  0.1375],
         [ 0.2408, -1.6463],
         [ 0.4673,  1.4432],
         [ 0.2025,  0.8386],
         [ 1.0679, -0.5033],
         [ 0.2402,  0.1207]]),
 tensor([[ 0.4275],
         [-5.1873],
         [ 7.3678],
         [ 4.9773],
         [ 1.2979],
         [10.2741],
         [ 0.2203],
         [ 1.7522],
         [ 8.0238],
         [ 4.2663]])]

In [4]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
loss = nn.MSELoss()

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for x, y in data_iter:
        l = loss(net(x), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000260
epoch 2, loss 0.000104
epoch 3, loss 0.000103


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 5.5695e-04, -8.1062e-06])
b的估计误差： tensor([-0.0003])
